<a href="https://colab.research.google.com/github/zlximon/Tensorflow_DL/blob/master/NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#downloading helper functions 
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py


--2022-06-16 08:40:40--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-06-16 08:40:40 (79.0 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [2]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys



#get text dataset

In [3]:
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

--2022-06-16 08:40:43--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.6.128, 142.250.159.128, 142.251.120.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.6.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2022-06-16 08:40:44 (112 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



  #visualizing text dataset

In [4]:
import pandas as pd 

In [5]:
train_df=pd.read_csv("train.csv")
test_df=pd.read_csv("test.csv")
train_df.head()
# 1 is for disaster 

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
train_df_shuffled=train_df.sample(frac=1,random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [9]:
len(train_df),len(test_df)

(7613, 3263)

In [10]:
import random
random_index = random.randint(0, len(train_df)-5) # create random indexes not higher than the total number of samples
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 1 (real disaster)
Text:
Udhampur terror attack: Militants attack police post 2 SPOs injured http://t.co/zMWeCBWVaO

---

Target: 0 (not real disaster)
Text:
The once desolate valley was transformed into a thriving hub of hiÛÓtech business.

---

Target: 0 (not real disaster)
Text:
if firefighters acted like cops they'd drive around shooting a flamethrower at burning buildings

---

Target: 1 (real disaster)
Text:
@dirk_trossen 

I've still got some of the snowstorm/hailstorm!

---

Target: 0 (not real disaster)
Text:
The last few days of summer are supposed to be the most fun so what's more fun then accidentally burning arm hair while playing w/ a lighter

---



#split data into training and validation  sets

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
train_sentences,val_sentences,train_labels,val_labels=train_test_split(train_df_shuffled["text"].to_numpy(),train_df_shuffled["target"],test_size=0.1,
                                                                       random_state=42)

In [13]:
len(train_sentences),len(val_sentences),len(train_labels),len(val_labels)

(6851, 762, 6851, 762)

In [14]:
train_sentences[:10],train_labels[:10 ]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), 5921    0


Embedding : create a matrix of features vector for each token 

#text vectorization ( tokenization)

In [15]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [16]:
text_vectorizer=TextVectorization(max_tokens=None# how many word in vocalubary  and None means there is no cap 
                                  ,standardize="lower_and_strip_punctuation",# turn all words to lower case and strip 
                                  split="whitespace",ngrams=None,output_mode="int",output_sequence_length=None,# how long do we want sequences be 
                                 )# pad_to_max_tokens=True)#add zeros into end of short lenght 

In [17]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization parameters
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (automatically add <OOV>)
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None # how long do you want your sequences to be?
                                    )

In [18]:
#find the average numbers of tokens
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))


15

In [19]:
# Setup text vectorization with custom variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [20]:
# fit the text vectorization to the training text
text_vectorizer.adapt(train_sentences)


In [21]:
#create a sample 
sample="There is a flood in my street"
text_vectorizer([sample])


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 74,   9,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [22]:
#choose a random sentence from training dataset
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
        \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
Conning Builds Strong Case for Portfolio #Diversification for Property-Casualty Insurers http://t.co/33FbR25t1O        

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   1, 6054, 1356,  823,   10, 9676,    1,   10, 3511, 2926,    1,
           0,    0,    0,    0]])>

In [23]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # the most common words in the vocab
bottom_5_words = words_in_vocab[-5:] # the least common words in the vocab
print(f"Most common words in vocab: {top_5_words}")
print(f"Least common words in vocab: {bottom_5_words}")

Most common words in vocab: ['', '[UNK]', 'the', 'a', 'in']
Least common words in vocab: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


#creating Embedding layer 

Embedding : turns positives integers into dense vectors of fixed size
parameters for our embedding

1-input_dim - The size of the vocabulary

2-output_dim - The size of the output embedding vector, for example, a value of 100 outputs a feature vector of size 100 for each word.

3-embeddings_initializer - How to initialize the embeddings matrix, default is "uniform" which randomly initalizes embedding matrix with uniform distribution. This can be changed for using pre-learned embeddings.

4- input_length - Length of sequences being passed to embedding layer.


In [24]:
from tensorflow.keras import layers 
embedding =layers.Embedding(input_dim=max_vocab_length,output_dim=128,input_length=max_length)
embedding

In [25]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nEmbedded version:")

# Embed the random sentence (turn it into numerical representation)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
Funds Needed for Rescued then Abandoned Cocker Spaniels http://t.co/RJrmW7nzy5      

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.02419863, -0.03738865, -0.02922716, ...,  0.04452557,
         -0.03385613,  0.02798079],
        [-0.04468142,  0.02357936, -0.00660106, ...,  0.00154377,
         -0.00947236, -0.03963993],
        [ 0.0470905 , -0.00105442,  0.04464425, ..., -0.03998788,
         -0.04384787,  0.03740704],
        ...,
        [ 0.02798799,  0.01613985, -0.0420678 , ..., -0.04293076,
         -0.02615732,  0.02814614],
        [ 0.02798799,  0.01613985, -0.0420678 , ..., -0.04293076,
         -0.02615732,  0.02814614],
        [ 0.02798799,  0.01613985, -0.0420678 , ..., -0.04293076,
         -0.02615732,  0.02814614]]], dtype=float32)>

In [26]:
sample_embed[0][0],sample_embed[0][0].shape,random_sentence[0]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.02419863, -0.03738865, -0.02922716,  0.02362892, -0.02947271,
         0.04230068,  0.02744284, -0.03341769, -0.03770201, -0.03632288,
         0.03243044,  0.02486891, -0.0049494 ,  0.00626539, -0.04532365,
        -0.00430524, -0.03222375,  0.00404979, -0.04995605, -0.03939305,
         0.03026021,  0.04162038,  0.04743712, -0.00720371, -0.02007816,
        -0.02587076, -0.00584672, -0.00309885, -0.01311877,  0.02093029,
         0.03523538, -0.04838648,  0.0021771 ,  0.04309553, -0.04066544,
         0.04261271, -0.04595398, -0.04047453, -0.03951031, -0.00963515,
        -0.03891604,  0.00278304, -0.00427844, -0.00324371,  0.0160399 ,
        -0.03903221, -0.02251747, -0.02110803, -0.01100048, -0.00785028,
        -0.01340906,  0.02618784, -0.04712973, -0.01226414, -0.00300692,
        -0.02087569,  0.02655656,  0.03747988, -0.01215848, -0.00260018,
         0.02297726, -0.02384456,  0.03185136,  0.01556659,  0.01100683,
  

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline


In [28]:
model0=Pipeline([
("tfidf",TfidfVectorizer()),#convert words to numbers using tfdif
("clf",MultinomialNB())#model the text
])
#fit pipeline to train data
model0.fit(train_sentences,train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [29]:
baseline_score=model0.score(val_sentences,val_labels)# evaluate 
baseline_score # accuracy 

0.7926509186351706

In [30]:
baseline_pred=model0.predict(val_sentences)
baseline_pred[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

#creating function to track and evaluate model 

In [31]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
  ----
  y_true = true labels in the form of a 1D array
  y_pred = predicted label in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall and f1-score between y_true and y_pred.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100 # get accuracy score in percentage value
  # Calculate model precision, recall and f1 score using "weighted" avergage
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  # Create a dictionary of model results
  model_results = {"accuracy": model_accuracy,
                   "precision": model_precision,
                   "recall": model_recall,
                   "f1": model_f1}
  return model_results

In [32]:
baseline_result=calculate_results(y_true=val_labels,y_pred=baseline_pred)
baseline_result

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

#model 1 (Dense model)

In [34]:
from helper_functions import create_tensorboard_callback


In [40]:
SAVE_DIR = "model_logs"


In [60]:
from tensorflow.keras import layers
inputs=(layers.Input(shape=(1,),dtype=tf.string))
x=text_vectorizer(inputs)
x=embedding(x)
x=layers.GlobalAveragePooling1D()(x)
#x=layers.GlobalMaxPool1D()(x)
outputs=layers.Dense(1,activation="sigmoid")(x)
model1=tf.keras.Model(inputs,outputs,name="model1")

In [61]:
model1.summary()

Model: "model1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d_1   (None, 128)              0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non-trai

In [50]:
model1.summary()

Model: "model1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_2 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non-trai

In [62]:
#compile model
model1.compile(loss=tf.keras.losses.binary_crossentropy,optimizer=tf.keras.optimizers.Adam(),metrics=["accuracy"])


In [63]:
#fit the model
model1_history=model1.fit(x=train_sentences,y=train_labels,epochs=5,validation_data=(val_sentences,val_labels),callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR, 
                                                                     experiment_name="simple_dense_model")])

Saving TensorBoard log files to: model_logs/simple_dense_model/20220616-092115
Epoch 1/5
215/215 [==============================] - 2s 5ms/step - loss: 0.5014 - accuracy: 0.8399 - val_loss: 0.4851 - val_accuracy: 0.7861
Epoch 2/5
215/215 [==============================] - 1s 4ms/step - loss: 0.3113 - accuracy: 0.9007 - val_loss: 0.4518 - val_accuracy: 0.7953
Epoch 3/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2450 - accuracy: 0.9169 - val_loss: 0.4555 - val_accuracy: 0.7940
Epoch 4/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2070 - accuracy: 0.9288 - val_loss: 0.4703 - val_accuracy: 0.7835
Epoch 5/5
215/215 [==============================] - 1s 4ms/step - loss: 0.1802 - accuracy: 0.9396 - val_loss: 0.4909 - val_accuracy: 0.7861


In [53]:
model1.evaluate(val_sentences,val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4715 - accuracy: 0.7861


[0.47153109312057495, 0.7860892415046692]

In [64]:
model1_pred_probs=model1.predict(val_sentences)
model1_pred_probs.shape

(762, 1)

In [65]:
model1_pred_probs[0]

array([0.2887181], dtype=float32)

In [71]:
#Convert model prediction probabilites to label format 
model1_preds=tf.squeeze(tf.round(model1_pred_probs))
model1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [73]:
model1_result=calculate_results(y_true=val_labels,y_pred=model1_preds)
model1_result


{'accuracy': 78.60892388451444,
 'f1': 0.7830068921982195,
 'precision': 0.7911704681403405,
 'recall': 0.7860892388451444}

In [75]:
baseline_result

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

In [ ]:
import numpy as np 
mp